In [27]:
#check the directory
using PyCall
using Pkg
PyCall.libpython

"/Users/zhengpingfeng/.julia/conda/3/lib/libpython3.9.dylib"

In [40]:
#set directory
Pkg.add("Conda")
Pkg.add("PyCall")
ENV["PYTHON"] ="/usr/bin/python3"
Pkg.build("PyCall")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/6e47d11ea2776bc5627421d59cdcc1296c058071/build.log`
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/1fc929f47d7c151c839c5fc1375929766fb8edcc/build.log`
Precompiling project...
  ✓ PyCall
  1 dependency successfully precompiled in 2 seconds (288 already precompiled, 1 skipped during auto due to previous errors)
  1 dependency precompiled but a different version is currently loaded. Restart julia to access the new version


In [48]:
#get xcs file
a = pyimport("xcs")

PyObject <module 'xcs' from '/Users/zhengpingfeng/.julia/conda/3/lib/python3.9/site-packages/xcs/__init__.py'>

In [49]:
#set variables
ScenarioObserver=a.scenarios.ScenarioObserver 
MUXProblem=a.scenarios.MUXProblem
PreClassifiedData=a.scenarios.PreClassifiedData
XCSAlgorithm=a.XCSAlgorithm
BitString=a.bitstrings
Scenario=a.scenarios 

PyObject <module 'xcs.scenarios' from '/Users/zhengpingfeng/.julia/conda/3/lib/python3.9/site-packages/xcs/scenarios.py'>

In [52]:
#set model
smodel = XCSAlgorithm()

PyObject <xcs.XCSAlgorithm object at 0x165383100>

In [53]:
smodel.exploration_probability = .1
smodel.discount_factor = 0
smodel.do_ga_subsumption = true
smodel.do_action_set_subsumption = true

true

In [54]:
#output the model using py style
py"""
def outputmodel(x):
    import logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    logger.info('Classifiers:\n\n%s\n', x)

"""

In [104]:
#load the data
#import the packet
using Plots
using ProgressBars
using Flux
using Flux: onehot
using Flux: Data.DataLoader
using Flux: onehotbatch, onecold, crossentropy
using Flux: @epochs
using Statistics
using MLDatasets
using CSV
using DataFrames
using Random:shuffle
using EvalMetrics
using Distributions
using DataFrames
using Random
using StatsBase

In [84]:
#load the data
df_red = CSV.read("winequality-red.csv",DataFrame)
df_white = CSV.read("winequality-white.csv",DataFrame)

#tag the wine 0 as red and 1 as white
insertcols!(df_red,13,:class => 0,makeunique=true)  
insertcols!(df_white,13,:class => 1,makeunique=true)  

,fixedacidity,volatileacidity,citricacid,residualsugar,chlorides,freesulfurdioxide
,Float64,Float64,Float64,Float64,Float64,Float64
1,7.0,0.27,0.36,20.7,0.045,45.0
2,6.3,0.3,0.34,1.6,0.049,14.0
3,8.1,0.28,0.4,6.9,0.05,30.0
4,7.2,0.23,0.32,8.5,0.058,47.0
5,7.2,0.23,0.32,8.5,0.058,47.0
6,8.1,0.28,0.4,6.9,0.05,30.0
7,6.2,0.32,0.16,7.0,0.045,30.0
8,7.0,0.27,0.36,20.7,0.045,45.0
9,6.3,0.3,0.34,1.6,0.049,14.0


In [90]:
#make the matrix
df_red_matrix = df_red |> Tables.matrix
df_white_matrix = df_white |> Tables.matrix

1599×13 Matrix{Float64}:
 7.0  0.27  0.36  20.7   0.045  45.0  …  1.001   3.0   0.45   8.8  6.0  1.0
 6.3  0.3   0.34   1.6   0.049  14.0     0.994   3.3   0.49   9.5  6.0  1.0
 8.1  0.28  0.4    6.9   0.05   30.0     0.9951  3.26  0.44  10.1  6.0  1.0
 7.2  0.23  0.32   8.5   0.058  47.0     0.9956  3.19  0.4    9.9  6.0  1.0
 7.2  0.23  0.32   8.5   0.058  47.0     0.9956  3.19  0.4    9.9  6.0  1.0
 8.1  0.28  0.4    6.9   0.05   30.0  …  0.9951  3.26  0.44  10.1  6.0  1.0
 6.2  0.32  0.16   7.0   0.045  30.0     0.9949  3.18  0.47   9.6  6.0  1.0
 7.0  0.27  0.36  20.7   0.045  45.0     1.001   3.0   0.45   8.8  6.0  1.0
 6.3  0.3   0.34   1.6   0.049  14.0     0.994   3.3   0.49   9.5  6.0  1.0
 8.1  0.22  0.43   1.5   0.044  28.0     0.9938  3.22  0.45  11.0  6.0  1.0
 8.1  0.27  0.41   1.45  0.033  11.0  …  0.9908  2.99  0.56  12.0  5.0  1.0
 8.6  0.23  0.4    4.2   0.035  17.0     0.9947  3.14  0.53   9.7  5.0  1.0
 7.9  0.18  0.37   1.2   0.04   16.0     0.992   3.18  0.63  10

In [111]:
#pick train and test data
cutdata = randsubseq(1:1599, 0.7)
train_data = vcat(df_red_matrix[cutdata,:], df_white_matrix[cutdata,:])
uncutdata = [i for i in 1:1599 if isempty(searchsorted(cutdata, i))]
test_data = vcat(df_red_matrix[uncutdata,:], df_white_matrix[uncutdata,:])

train_x = train_data[:,1:12]
test_x = test_data[:,1:12]
train_y = train_data[:,13]
test_y = test_data[:,13]

964-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [107]:
scenarioTrain = PreClassifiedData(train_x,train_y)
scenarioTest = PreClassifiedData(test_x,test_y)

PyObject <xcs.scenarios.PreClassifiedData object at 0x178029fd0>

In [108]:
model_train = smodel.new_model(scenarioTrain)
model_test = smodel.new_model(scenarioTest)

PyObject <xcs.ClassifierSet object at 0x1780e72e0>

In [109]:
#run the model
model_train = smodel.run(scenarioTrain)
model_test = smodel.run(scenarioTest)

PyObject <xcs.ClassifierSet object at 0x165364df0>

In [110]:
#train model output
py"outputmodel"(model_train)

INFO:__main__:Classifiers:

11#####11#1# => 0.0
    Time Stamp: 2179
    Average Reward: 6.850410822761393e-05
    Error: 0.000606261357814383
    Fitness: 0.010253872839180828
    Experience: 94
    Action Set Size: 93.52847680423079
    Numerosity: 1
111#11##111# => 0.0
    Time Stamp: 2179
    Average Reward: 6.850410822761393e-05
    Error: 0.000606261357814383
    Fitness: 0.010253873796648535
    Experience: 307
    Action Set Size: 93.52847643334148
    Numerosity: 1
1###1##1111# => 0.0
    Time Stamp: 2179
    Average Reward: 6.850410822761393e-05
    Error: 0.000606261357814383
    Fitness: 0.010253873822106376
    Experience: 410
    Action Set Size: 93.5284763387548
    Numerosity: 1
11##11#1#1## => 0.0
    Time Stamp: 2179
    Average Reward: 6.850410822761393e-05
    Error: 0.000606261357814383
    Fitness: 0.010253873822106376
    Experience: 445
    Action Set Size: 93.5284763387548
    Numerosity: 1
11##1##11#1# => 0.0
    Time Stamp: 2179
    Average Reward: 6.85041082

In [112]:
#test model output
py"outputmodel"(model_test)

INFO:__main__:Classifiers:

1101#1#111#1 => 1.0
    Time Stamp: 581
    Average Reward: 0.6249964799277179
    Error: 0.09378854479148822
    Fitness: 0.014260181256477709
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
#1###1##1111 => 0.0
    Time Stamp: 938
    Average Reward: 0.0004093628563497568
    Error: 0.0029474125657182482
    Fitness: 0.01870887329237043
    Experience: 166
    Action Set Size: 41.37873076804619
    Numerosity: 1
11#11111111# => 1.0
    Time Stamp: 937
    Average Reward: 1.0
    Error: 0.0
    Fitness: 0.02128274686883024
    Experience: 99
    Action Set Size: 46.98699037524803
    Numerosity: 1
#1#1111#1111 => 1.0
    Time Stamp: 937
    Average Reward: 1.0
    Error: 0.0
    Fitness: 0.021282748626781478
    Experience: 198
    Action Set Size: 46.98699016859669
    Numerosity: 1
#1111#11111# => 1.0
    Time Stamp: 937
    Average Reward: 1.0
    Error: 0.0
    Fitness: 0.02128274862678175
    Experience: 166
    Action Set Size: 46.9869901686